[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/tabular-classification/sklearn/iris-classifier/iris-tabular-sklearn.ipynb)


# Iris classification using sklearn

This notebook illustrates how sklearn models can be upladed to the Unbox platform.

## Importing the modules and loading the dataset

In [32]:
import numpy as np

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [33]:
iris = datasets.load_iris()
X = iris.data[:, 0:2]  # we only take the first two features for visualization
y = iris.target

## Splitting the data into training and validation sets

In [34]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Training and evaluating the model's performance

In [35]:
sklearn_model = LogisticRegression(random_state=1300)
sklearn_model.fit(x_train, y_train)

LogisticRegression(random_state=1300)

In [36]:
print(classification_report(y_val, sklearn_model.predict(x_val)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.73      0.62      0.67        13
           2       0.38      0.50      0.43         6

    accuracy                           0.73        30
   macro avg       0.70      0.71      0.70        30
weighted avg       0.76      0.73      0.74        30



## Unbox part!

### Instantiating the client

In [37]:
import unboxapi

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### Creating a project

In [38]:
project = client.create_project(name="Iris Prediction", 
                                description="Evaluation of ML approaches to predict the iris")

Creating project on Unbox! Check out https://unbox.ai/projects to have a look!


### Uploading the validation set

In [44]:
import pandas as pd

feature_names = iris.feature_names[:2]
class_names = iris.target_names.tolist()

df_train = pd.DataFrame(x_train, columns=feature_names)
df_train["target"] = y_train
df_val = pd.DataFrame(x_val, columns=feature_names)
df_val["target"] = y_val

In [45]:
from unboxapi.tasks import TaskType

dataset = project.add_dataframe(
    df=df,
    class_names=class_names,
    label_column_name='target',
    name="Iris Sklearn Train",
    description='this is my iris training demo dataset',
    task_type=TaskType.TabularClassification,
    feature_names=feature_names,
)

Uploading dataset to Unbox! Check out https://unbox.ai/datasets to have a look!


### Uploading the model

First, it is important to create a `predict_proba` function, which is how Unbox interacts with your model

In [46]:
def predict_proba(model, input_features: np.ndarray):
    return model.predict_proba(input_features)

Let's test the `predict_proba` function to make sure the input-output format is consistent with what Unbox expects:

In [47]:
predict_proba(sklearn_model, df[:3][feature_names].to_numpy())

array([[0.07912881, 0.59599725, 0.32487394],
       [0.00559202, 0.67039365, 0.32401432],
       [0.96339062, 0.01875935, 0.01785003]])

Now, we can upload the model:

In [48]:
from unboxapi.models import ModelType

model = project.add_model(
    function=predict_proba, 
    model=sklearn_model,
    model_type=ModelType.sklearn,
    task_type=TaskType.TabularClassification,
    class_names=class_names,
    name='Iris - Logistic reg.',
    description='this is my first tabular classification model',
    feature_names=feature_names,
    train_sample_df=df_train,
    train_sample_label_column_name='target',
)

Bundling model and artifacts...
Uploading model to Unbox! Check out https://unbox.ai/models to have a look!
